## Kivent Tutorial
### Part 1: Setting up and App
https://github.com/kivy/kivent/wiki/Getting-Started-1:-Setting-Up-an-App

Start with the boilerplate debug code. 

In [ ]:
%%file debug.kv
#:import random random
    
<Widget>:
    canvas.after:
        Color:
            rgba: 1,1,1,.5
        Line:
            rectangle: self.x, self.y, self.width, self.height
            width: 2
                
<DebugLabel@Button>:
    size: self.parent.size
    pos: self.parent.pos
    background_color: random.random(), random.random(), random.random(), 0.6
    text: 'debuglabel'

Now make a default kivy app consisting of `main.py` and `yourappname.kv`. (Note the reference to `debug.kv` in `yourappname.kv` and the debug label).

In [ ]:
%%file main.py
from kivy.app import App
from kivy.uix.widget import Widget

class TestGame(Widget):
    pass

class YourAppNameApp(App):
    def build(self):
        pass
    
if __name__ == '__main__':
    YourAppNameApp().run()

In [ ]:
%%file yourappname.kv
#:include debug.kv
#:kivy 1.9.0

TestGame:

<TestGame>:
    DebugLabel:
        text: "testgame"

In [ ]:
# run your app
!python main.py

Now check out two different ways to do the **exact same thing** with different instantiation code. 

In [ ]:
%%file main.py
from kivy.app import App
from kivy.uix.widget import Widget

class TestGame(Widget):
    pass

class YourAppNameApp(App):
    def build(self):
        return TestGame() ## change this from pass
    
if __name__ == '__main__':
    YourAppNameApp().run()

In [ ]:
%%file yourappname.kv
#:include debug.kv
#:kivy 1.9.0

## Deleted the refernce to TestGame as the root widget

<TestGame>:
    DebugLabel:
        text: "testgame"

In [ ]:
# run your app. you should get the same thing. 
!python main.py

In [ ]:
%%file main.py
from kivy.app import App
from kivy.uix.widget import Widget

# The TestGame class is now completely defined in the .kv file

class YourAppNameApp(App):
    def build(self):
        pass
    
if __name__ == '__main__':
    YourAppNameApp().run()

In [ ]:
%%file yourappname.kv
#:include debug.kv
#:kivy 1.9.0

TestGame:
    
<TestGame@Widget>: ## adding makes TestGame a Widget class
    DebugLabel:
        text: "testgame"

In [ ]:
# run it again
!python main.py

Now in pure python with no .kv file...hmmmm...later. 

## Time to add some kivent
We'll go back to the first version of default app. 

In [ ]:
%%file main.py
from kivy.app import App
from kivy.uix.widget import Widget
import kivent_core ## add this import statement to kivent

class TestGame(Widget):
    pass

class YourAppNameApp(App):
    def build(self):
        pass
    
if __name__ == '__main__':
    YourAppNameApp().run()

First add the `GameWorld` Widget that will manage the game data.

In [ ]:
%%file yourappname.kv
#:include debug.kv
#:kivy 1.9.0

TestGame:

<TestGame@Widget>: ## adding @Widget makes this the root widget
    gameworld: _gameworld  ##reference to GameWorld id
    DebugLabel:
        text: "testgame"
    GameWorld:
        id: _gameworld
        size_of_gameworld: 100*1024 ## required see explanation
        size_of_entity_block: 128 ## required see explanation
        system_count: 8 ## max number of GameSystems with components
        zones: {'general':10000}
        DebugLabel:
            text: "gameworld"

In [ ]:
#!python main.py

In [ ]:
%load yourappname.kv

In [ ]:
%load main.py